In [1]:
# import pandas as pd

In [2]:
# df = pd.read_csv('cases.csv')
# df.head()

In [3]:
# M = df['cases'].to_numpy().reshape(-1, 1)

In [4]:
import numpy as np
from numf_func import get_peaks, numf

np.random.seed(42)

In [5]:
# pvals = get_peaks(M, nrad=2)

In [8]:
m = 20
n = 8
r = 5

M = np.random.rand(m, n)
m, n = M.shape
W0 = np.random.rand(m, r)
H0 = np.random.rand(r, n)

In [10]:
%%timeit
W, H  = numf(M, W0, H0, iters=5)

75 ms ± 6.66 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
print(np.linalg.norm(M - W @ H, 'fro') / np.linalg.norm(M, 'fro'))

NameError: name 'W' is not defined